In [ ]:
##in this version i will say screw pipelines and just do it without for now ... 
#maybe I will see what this infinity issue is if I can inspect the resuly of each step manually

In [ ]:
##Round 1 cleaning complete. v1 decisions: keep all nan as new feature/value; no skew or outlier action has been taken
##Round 1 exploration complete. (a) Distribution of each feature explored (b) feature-target and feature-feature relationships explored 

##Round 1 modelling Plan: make one model for each notebook, try to do systematic tuning ... take best few models (?) or is it too soon?
##then go back and do feature engineering and feature selection based on these base-line models
##Build pipe-line: dummies, scaling, tuning ...

##next steps:
##
##Flask API Deployment
##build small webpage interaction


### Tuning of KNN 

Main parameters: k = num_neighbors, and metric

In [ ]:
## following https://machinelearningmastery.com/hyperparameters-for-classification-machine-learning-algorithms/

In [ ]:
##brief review of KNN vs K-means ... 
##K-means (unspervised) has no labels, just feature space ... you create k-labels (choice of k is tricky and important)
##then run thru k-means clustering algorithm to assign k-labels (clusters) to the data 
##b
##KNN (supervised) is simpler because data already comes with p-labels (the role of k is different here) 
##in the training data on the test data you simply look at the k-nearet neighors (of training set) and take majority vote 
##on what label to assign to new point. The choice of k is important and the metric can change the outcomes.

In [7]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer, OrdinalEncoder
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.decomposition import PCA 
from sklearn.linear_model import Ridge
from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, f1_score
import seaborn as sns

import pickle

#sns.set()
sns.set(rc={'figure.figsize':(8,5)})

#! very useful 

pd.set_option('display.max_columns', 500)


In [8]:
##baseline models
from sklearn.model_selection import train_test_split # For train/test splits
from sklearn.neighbors import KNeighborsClassifier # The k-nearest neighbor classifier
from sklearn.feature_selection import VarianceThreshold # Feature selector
from sklearn.pipeline import Pipeline # For setting up pipeline
# Various pre-processing steps
from sklearn.preprocessing import Normalizer, StandardScaler, MinMaxScaler, PowerTransformer, MaxAbsScaler, LabelEncoder

from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV

from sklearn.model_selection import GridSearchCV # For optimization


In [9]:

from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV

from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier

#rfc = RandomForestClassifier(max_depth=4, random_state=0)
#gnb = GaussianNB()
#lgc = LogisticRegression()


In [10]:
## cleaning tool


def perc_null(X):
    
    total = X.isnull().sum().sort_values(ascending=False)
    data_types = X.dtypes
    percent = (X.isnull().sum()/X.isnull().count()).sort_values(ascending=False)

    missing_data = pd.concat([total, data_types, percent], axis=1, keys=['Total','Type' ,'Percent'])
    return missing_data


In [11]:
#general functions ... testing done in notebook Proj1_Loans_Test_Imputers_Scalers

def most_freq_imputer(df, col_list): #data =df, list_col = [col1, col2, ...] columns of df
    for col in col_list:
        most_freq_val = df[col].value_counts().index[0]  #0 grabs most frequent
        df.loc[df[col].isnull(), col] = most_freq_val


def mean_imputer(df, col_list): #data =df, list_col = [col1, col2, ...] columns of df
    for col in col_list:
        mean = round(np.mean(df[col]),2)
        df.loc[df[col].isnull(), col] = mean



In [4]:
#max_min scaler

def max_min_helper(x, min, max, new_min =0, new_max=1):
    y = (x-min)/(max-min)   #scales to min = 0, max = 1
    scaled = y*(new_max-new_min) + new_min 
    return scaled
#scale by max 

def max_min_scaler(df, col_list, new_min =0, new_max=1):
    for col in col_list:

        min = np.min(df[col])
        max = np.max(df[col])
        
        df[col] = df[col].map(lambda x: max_min_helper(x, min, max, new_min, new_max))
                   
    
##why is this one not working? because you had max, min arguments switched in your function call

In [5]:
#mean scaler


def mean_scaler(df, col_list):
    for col in col_list:
        mean = np.mean(df[col])
        sigma = np.std(df[col]) #std deviation
        df[col] = (df[col] - mean)/sigma
              

In [6]:
path = "/Users/joejohns/Downloads/data_loans.csv"

data = pd.read_csv(path) 
print(data.shape)
data.head(3)

NameError: name 'pd' is not defined

## data processing from round 1


In [ ]:
perc_null(data)

    

In [ ]:
nan_feats = list(perc_null(data)[ perc_null(data)['Total'] >0].index) 
#nan_feats

In [ ]:
data.rename(columns = {'ApplicantIncome':'Applicant_Income', 'CoapplicantIncome': 'Coapplicant_Income', 'LoanAmount':'Loan_Amount'}, inplace = True)

In [ ]:
target = ['Loan_Status']
ID_feats = ['Loan_ID',] 

#feature types (this must be done manually ... although data types would give a pretty good approximation)

numer_feats = ['Applicant_Income', 'Coapplicant_Income', 'Loan_Amount', 'Loan_Amount_Term',]

ord_cat_feats = [ 'Dependents', 'Education', ]
nom_cat_feats = ['Gender', 'Married', 'Self_Employed', 'Credit_History', 'Property_Area',]



In [ ]:
##fix ordinal cats (write general function which does this, no ordinal encoder)
for item in ['0', '1', '2', '3+']:
    if item != '3+':
        i = int(item)
    else:
        i = 3
    data.loc[data['Dependents'] == item,'Dependents']  = i


In [ ]:
#fix target; make it numeric; useful for regression modelling

data.loc[data["Loan_Status"] == 'Y', "Loan_Status"] = 1
data.loc[data["Loan_Status"] == 'N', "Loan_Status"] = 0



In [ ]:
#fix this nom cat ... float --> string

data.loc[data['Credit_History'] ==0.0, 'Credit_History'] = 'No'
data.loc[data['Credit_History'] == 1.0, 'Credit_History'] = 'Yes'  #I use capitalized "Yes", "No" to be conistent with their other entries

In [ ]:
#turn this into ordinal cat

data.loc[data['Education']=='Not Graduate', "Education"] = 0
data.loc[data['Education']=='Graduate', "Education"] = 1

In [ ]:
impute_most_freq =[]

for feat in numer_feats:
    perc_val = data[feat].value_counts()/data.shape[0]
    if perc_val.iloc[0] >=0.7:
        impute_most_freq+=[feat]
    


In [ ]:
impute_most_freq

In [ ]:


    



#set up some feature lists for imputing and scaling 

#steps:

#1. create new cat recording if NaN in specified numeric or nominal features 
#(do this before imputing of course)

#numeric and nominal: 
#2. impute most_freq (Loan_Term)
#3. impute mean (everyone else)
#4. scale by max, min (or by mean; try both)

#nominal cats:
#5. get_dummies, nan yes
#6. get_dummies, nan no



In [ ]:



A = set(ord_cat_feats+numer_feats) 
B = set(nan_feats)
C = set(impute_most_freq)
impute_mean = list(A-C)

#impute_most_freq = ['Loan_Amount_Term']  #how can we identify this automatically? It's a cts var where over X% of values = one value 
#impute_mean = [x for x in ord_cat_feats+numer_feats if x not in impute_most_freq] 
 
scale = ord_cat_feats+numer_feats

#can make a list comprehension (rather than loop) that extracts these ... 
numer_ord_nan_new_cat = [ 'Loan_Amount', 'Loan_Amount_Term', 'Dependents', ] #make new category for these 'unk' = yes/no; ##impute by mean
nom_cat_nan_dummies = ['Credit_History', 'Self_Employed', 'Gender', 'Married',]  ##make new dumies cat_val for nan
nom_cat_dummies = ['Property_Area','missing_data']  #regular dummies



In [ ]:
impute_mean

In [ ]:
##carry out imputation and scaling steps

In [ ]:
#step 1. create new categorical (records if any null values in any of specified cats)

data['missing_data'] = 'No'  
for feature in numer_ord_nan_new_cat:
    data.loc[(data[feature].isnull()) , 'missing_data'] = 'Yes'

    
nom_cat_feats+=['missing_data']

In [ ]:
##now we execute all imputing and scaling ... bruh


#steps:

#1. DONE create new cat recording if NaN in specified numeric or nominal features 
#(do this before imputing of course)

#numeric and nominal: 
#2. impute most_freq (Loan_Term)
#3. impute mean (everyone else)
#4. scale by max, min (or by mean; try both)

#nominal cats:
#5. get_dummies, nan yes
#6. get_dummies, nan no

perc_null(data)

In [ ]:
most_freq_imputer(data, impute_most_freq)
perc_null(data)

In [ ]:
mean_imputer(data, impute_mean)
perc_null(data)

In [ ]:
data = pd.get_dummies(data, columns = nom_cat_nan_dummies, dummy_na =True)
perc_null(data)

In [ ]:
data = pd.get_dummies(data, columns =nom_cat_dummies, dummy_na =False)
perc_null(data)

In [ ]:
data_mean = data.copy()
data_norm = data.copy()
mean_scaler(data_mean, scale)
max_min_scaler(data_norm, scale)


In [ ]:
model_feats = list(set(perc_null(data_mean).index) - {'Loan_ID','Loan_Status',})


In [ ]:
#let me now follow https://machinelearningmastery.com/modeling-pipeline-optimization-with-scikit-learn/
#which does cv = 3 instead of cv = stratififedkfold( ... )


y = data_mean.loc[:, target]
X= data_mean.loc[:, model_feats]
y = y.astype(int)
# Split the data into test and train
X_train, X_test, y_train, y_test = train_test_split(
    X,  
    y, 
    test_size=1/3,
    random_state=0)
 
print(X_train.shape)
print(X_test.shape)

#np.array(X)

In [ ]:
X_train

In [ ]:
knn = KNeighborsClassifier()

knn.fit(X_train, np.ravel(y_train))

In [ ]:
knn.score(X_test, np.ravel(y_test))  #77% ? shitty! ha ha was 80% in my original version

In [ ]:
##Sat Sept 18, 1:30 pm ... make pipe line? or do tuning and feature selection? 

In [ ]:

from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(max_depth=4, random_state=0)
gnb = GaussianNB()
lgc = LogisticRegression()


In [ ]:

rfc.fit(X_train, np.ravel(y_train))

In [ ]:
rfc.score(X_test, np.ravel(y_test))  ##aha! I used rfc before and it IS improved to 82% ... cool. That makes me feel better ... (all that effort should not make it worse)

In [ ]:

lgc.fit(X_train, np.ravel(y_train))

In [ ]:
lgc.score(X_test, np.ravel(y_test))  ##aha! I used rfc before and it IS improved to 82% ... cool. That makes me feel better ... (all that effort should not make it worse)

In [ ]:

knn = pipeline.fit(X_train, y_train)
print('Training set score: ' + str(knn.score(X_train,y_train)))
print('Test set score: ' + str(knn.score(X_test,y_test)))


In [ ]:
what ... the fuck... it seems there is an infinite value (or too large)

In [ ]:

class DataframeTransformer():
    def __init__(self, func):
        self.func = func

    def transform(self, input_df, **transform_params):
        return self.func(input_df)

    def fit(self, X, y=None, **fit_params):
        return self

# this function takes a dataframe as input and
# returns a modified version thereof
def get_dummies_dataframe(input_df):
    return pd.get_dummies(input_df, columns = nom_cat_feats)


# this pipeline has a single step
pipeline = Pipeline([
    ("get_dummies", DataframeTransformer(get_dummies_dataframe))
])